#This Project is developed to take a wav file of a single voice of music and get the musical pitch at any time frame.

Imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import wave
import sys
import statistics
import random
import math
import librosa

Name of User (used in application)

In [ ]:
userId = 'Temp'

File location (only accepts .wav file)